<div style="text-align: center">
  <img src="https://github.com/KarolisRam/MineRL2021-Intro-baselines/blob/main/img/colab_banner.png?raw=true">
</div>

# Introduction
This notebook contains the Behavioural Cloning baselines for the Research track of the [MineRL 2021](https://minerl.io/) competition. To run it you will need to enable GPU by going to `Runtime -> Change runtime type` and selecting GPU from the drop down list.

These baselines differ slightly from the standalone version of these baselines on github - the DATA_SAMPLES parameter is set to 400,000 instead of the default 1,000,000. This is done to fit into the RAM limits of Colab.

To train the agent using the obfuscated action space we first discretize the action space using KMeans clustering. We then train the agent using Behavioural cloning. The training takes 10-15 mins.

You can find more details about the obfuscation here:  
[K-means exploration](https://minerl.io/docs/tutorials/k-means.html)

Also see the in-depth analysis of the obfuscation and the KMeans approach done by one of the teams in the 2020 competition:

[Obfuscation and KMeans analysis](https://github.com/GJuceviciute/MineRL-2020)

Please note that any attempt to work with the obfuscated state and action spaces should be general and work with a different dataset or even a completely new environment.

# Setup

In [1]:
%%capture
!sudo add-apt-repository -y ppa:openjdk-r/ppa
!sudo apt-get purge openjdk-*
!sudo apt-get install openjdk-8-jdk
!sudo apt-get install xvfb xserver-xephyr vnc4server python-opengl ffmpeg

In [2]:
%%capture
!pip install --upgrade 'setuptools==57.5.0'
!pip install 'gym==0.19.0'


In [3]:
!pip install --upgrade wheel==0.38.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: wheel
    Found existing installation: wheel 0.40.0
    Uninstalling wheel-0.40.0:
      Successfully uninstalled wheel-0.40.0


In [4]:
#%%capture
!pip install --upgrade minerl
!pip install pyvirtualdisplay
# !pip install pytorch
!pip install scikit-learn
!pip install -U colabgymrender

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 MB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached gym-0.19.0.tar.gz (1.6 MB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 221.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 13.5 MB/s eta 0:00:00
  Using cached cloudpickle-1.6.0-py3-none-any.whl (23 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 84.9 MB/s eta 0:00:00
  Created wheel for minerl: filename=minerl-0.4.4-cp310-cp310-linux_x86_64.whl size=563666091 sha256=aa118b58933d30a4cb8e792b2d7d3e79bd6d88a54d0d195f380cc8a25241dcc7
  Stored in directory: /root/

In [5]:
# # !apt update 
# !apt install -y python-numpy python-dev cmake zlib1g-dev libjpeg-dev xvfb ffmpeg xorg-dev python-opengl libboost-all-dev libsdl2-dev swig

# !pip3 install pyvirtualdisplay piglet gym torch torchvision
# # !pip3 install "gym[atari]"

In [6]:
!apt-get install xorg openbox
!apt-get install xvfb

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  apport apport-symptoms aspell aspell-en bubblewrap desktop-file-utils
  dictionaries-common dmidecode docbook-xml dosfstools eject emacsen-common
  enchant-2 gdisk giblib1 gnome-icon-theme gnome-terminal gnome-terminal-data
  gstreamer1.0-gl gstreamer1.0-plugins-base gstreamer1.0-plugins-good
  gstreamer1.0-pulseaudio gstreamer1.0-x gvfs gvfs-common gvfs-daemons
  gvfs-libs hunspell-en-us keyboard-configuration libaa1 libaspell15
  libatasmart4 libblockdev-crypto2 libblockdev-fs2 libblockdev-loop2
  libblockdev-part-err2 libblockdev-part2 libblockdev-swap2 libblockdev-utils2
  libblockdev2 libcdparanoia0 libdv4 libenchant-2-2 libgck-1-0 libgcr-base-3-1
  libgpgme11 libgraphene-1.0-0 libgstreamer-gl1.0-0
  libgstreamer-plugins-good1.0-0 libharfbuzz-icu0 libhunspell-1.7-0 libhyphen0
  libid3tag0 libimlib2 libjavascriptcoregtk-4.0-18 libloc

# Import Libraries

In [ ]:
import random
import numpy as np
import torch as th
from torch import nn
import gym
import minerl
from tqdm.notebook import tqdm
from colabgymrender.recorder import Recorder
from gym.wrappers import Monitor

from pyvirtualdisplay import Display
from sklearn.cluster import KMeans
import logging
logging.disable(logging.ERROR) # reduce clutter, remove if something doesn't work to see the error logs.

# Neural network

In [ ]:
class NatureCNN(nn.Module):
    """
    CNN from DQN nature paper:
        Mnih, Volodymyr, et al.
        "Human-level control through deep reinforcement learning."
        Nature 518.7540 (2015): 529-533.

    Nicked from stable-baselines3:
        https://github.com/DLR-RM/stable-baselines3/blob/master/stable_baselines3/common/torch_layers.py

    :param input_shape: A three-item tuple telling image dimensions in (C, H, W)
    :param output_dim: Dimensionality of the output vector
    """

    def __init__(self, input_shape, output_dim):
        super().__init__()
        n_input_channels = input_shape[0]
        self.cnn = nn.Sequential(
            nn.Conv2d(n_input_channels, 32, kernel_size=8, stride=4, padding=0),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=0),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.Flatten(),
        )

        # Compute shape by doing one forward pass
        with th.no_grad():
            n_flatten = self.cnn(th.zeros(1, *input_shape)).shape[1]

        self.linear = nn.Sequential(
            nn.Linear(n_flatten, 512),
            nn.ReLU(),
            nn.Linear(512, output_dim)
        )

    def forward(self, observations: th.Tensor) -> th.Tensor:
        return self.linear(self.cnn(observations))

# Setup training

In [9]:
def train():
    # For demonstration purposes, we will only use ObtainPickaxe data which is smaller,
    # but has the similar steps as ObtainDiamond in the beginning.
    # "VectorObf" stands for vectorized (vector observation and action), where there is no
    # clear mapping between original actions and the vectors (i.e. you need to learn it)
    data = minerl.data.make("MineRLObtainIronPickaxeVectorObf-v0",  data_dir='data', num_workers=1)

    # First, use k-means to find actions that represent most of them.
    # This proved to be a strong approach in the MineRL 2020 competition.
    # See the following for more analysis:
    # https://github.com/GJuceviciute/MineRL-2020

    # Go over the dataset once and collect all actions and the observations (the "pov" image).
    # We do this to later on have uniform sampling of the dataset and to avoid high memory use spikes.
    all_actions = []
    all_pov_obs = []

    print("Loading data")
    trajectory_names = data.get_trajectory_names()
    random.shuffle(trajectory_names)

    # Add trajectories to the data until we reach the required DATA_SAMPLES.
    for trajectory_name in trajectory_names:
        trajectory = data.load_data(trajectory_name, skip_interval=0, include_metadata=False)
        for dataset_observation, dataset_action, _, _, _ in trajectory:
            all_actions.append(dataset_action["vector"])
            all_pov_obs.append(dataset_observation["pov"])
        if len(all_actions) >= DATA_SAMPLES:
            break

    all_actions = np.array(all_actions)
    all_pov_obs = np.array(all_pov_obs)

    # Run k-means clustering using scikit-learn.
    print("Running KMeans on the action vectors")
    kmeans = KMeans(n_clusters=NUM_ACTION_CENTROIDS)
    kmeans.fit(all_actions)
    action_centroids = kmeans.cluster_centers_
    print("KMeans done")

    # Now onto behavioural cloning itself.
    # Much like with intro track, we do behavioural cloning on the discrete actions,
    # where we turn the original vectors into discrete choices by mapping them to the closest
    # centroid (based on Euclidian distance).

    network = NatureCNN((3, 64, 64), NUM_ACTION_CENTROIDS).cuda()
    optimizer = th.optim.Adam(network.parameters(), lr=LEARNING_RATE)
    loss_function = nn.CrossEntropyLoss()

    num_samples = all_actions.shape[0]
    update_count = 0
    losses = []
    # We have the data loaded up already in all_actions and all_pov_obs arrays.
    # Let's do a manual training loop
    print("Training")
    for _ in range(EPOCHS):
        # Randomize the order in which we go over the samples
        epoch_indices = np.arange(num_samples)
        np.random.shuffle(epoch_indices)
        for batch_i in range(0, num_samples, BATCH_SIZE):
            # NOTE: this will cut off incomplete batches from end of the random indices
            batch_indices = epoch_indices[batch_i:batch_i + BATCH_SIZE]

            # Load the inputs and preprocess
            obs = all_pov_obs[batch_indices].astype(np.float32)
            # Transpose observations to be channel-first (BCHW instead of BHWC)
            obs = obs.transpose(0, 3, 1, 2)
            # Normalize observations. Do this here to avoid using too much memory (images are uint8 by default)
            obs /= 255.0

            # Map actions to their closest centroids
            action_vectors = all_actions[batch_indices]
            # Use numpy broadcasting to compute the distance between all
            # actions and centroids at once.
            # "None" in indexing adds a new dimension that allows the broadcasting
            distances = np.sum((action_vectors - action_centroids[:, None]) ** 2, axis=2)
            # Get the index of the closest centroid to each action.
            # This is an array of (batch_size,)
            actions = np.argmin(distances, axis=0)

            # Obtain logits of each action
            logits = network(th.from_numpy(obs).float().cuda())

            # Minimize cross-entropy with target labels.
            # We could also compute the probability of demonstration actions and
            # maximize them.
            loss = loss_function(logits, th.from_numpy(actions).long().cuda())

            # Standard PyTorch update
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            update_count += 1
            losses.append(loss.item())
            if (update_count % 1000) == 0:
                mean_loss = sum(losses) / len(losses)
                tqdm.write("Iteration {}. Loss {:<10.3f}".format(update_count, mean_loss))
                losses.clear()
    print("Training done")

    # Save network and the centroids into separate files
    np.save(TRAIN_KMEANS_MODEL_NAME, action_centroids)
    th.save(network.state_dict(), TRAIN_MODEL_NAME)
    del data

# Parameters

In [10]:
# Parameters:
EPOCHS = 5  # how many times we train over dataset.
LEARNING_RATE = 0.0001  # Learning rate for the neural network.
BATCH_SIZE = 32
NUM_ACTION_CENTROIDS = 100  # Number of KMeans centroids used to cluster the data.

DATA_SAMPLES = 300000  # how many samples to use from the dataset. Impacts RAM usage

TRAIN_MODEL_NAME = 'research_potato.pth'  # name to use when saving the trained agent.
TEST_MODEL_NAME = 'research_potato.pth'  # name to use when loading the trained agent.
TRAIN_KMEANS_MODEL_NAME = 'centroids_for_research_potato.npy'  # name to use when saving the KMeans model.
TEST_KMEANS_MODEL_NAME = 'centroids_for_research_potato.npy'  # name to use when loading the KMeans model.

TEST_EPISODES = 10  # number of episodes to test the agent for.
MAX_TEST_EPISODE_LEN = 18000  # 18k is the default for MineRLObtainDiamondVectorObf.

# Download the data

In [15]:
minerl.data.download(directory='data', environment='MineRLObtainIronPickaxeVectorObf-v0');
# minerl.data.download(directory='data', environment='MineRLObtainIronPickaxe-v0');

Download: https://minerl.s3.amazonaws.com/v4/MineRLObtainIronPickaxeVectorObf-v0.tar: 3083.0MB [03:47, 13.55MB/s]                                


# Train

In [ ]:
# run only once!!!
display = Display(visible=0, size=(400, 300))
display.start();

In [16]:
train()  # only need to run this once.

Loading data


100%|██████████| 6283/6283 [00:00<00:00, 166451.78it/s]


Running KMeans on the action vectors
KMeans done
Training
Iteration 1000. Loss 2.247     
Iteration 2000. Loss 2.051     
Iteration 3000. Loss 1.970     
Iteration 4000. Loss 1.918     
Iteration 5000. Loss 1.862     
Iteration 6000. Loss 1.800     
Iteration 7000. Loss 1.772     
Iteration 8000. Loss 1.714     
Iteration 9000. Loss 1.691     
Iteration 10000. Loss 1.641     
Iteration 11000. Loss 1.616     
Iteration 12000. Loss 1.606     
Iteration 13000. Loss 1.563     
Iteration 14000. Loss 1.532     
Iteration 15000. Loss 1.532     
Iteration 16000. Loss 1.522     
Iteration 17000. Loss 1.485     
Iteration 18000. Loss 1.501     
Iteration 19000. Loss 1.437     
Iteration 20000. Loss 1.434     
Iteration 21000. Loss 1.410     
Iteration 22000. Loss 1.390     
Iteration 23000. Loss 1.392     
Iteration 24000. Loss 1.389     
Iteration 25000. Loss 1.364     
Iteration 26000. Loss 1.355     
Iteration 27000. Loss 1.358     
Iteration 28000. Loss 1.324     
Iteration 29000. Loss 1.326

# Start Minecraft

In [ ]:
!xvfb-run -s "-ac -screen 0 1280x1024x24" xvinfo

X-Video Extension version 2.2
screen #0
 no adaptors present


In [ ]:
# !rm -rf /content/video

In [ ]:
# !rm -rf /content/logs

In [17]:
env = gym.make('MineRLObtainIronPickaxe-v0')
# env = Monitor(env, './video', force=True)
# env = Recorder(env, './video', fps=60)

In [18]:
obs = env.reset()
obs

{'pov': array([[[  9,  14,   9],
         [ 14,  22,  14],
         [  0,   0,   0],
         ...,
         [118, 147, 213],
         [118, 147, 212],
         [118, 147, 212]],
 
        [[  9,  14,   9],
         [  9,  14,   9],
         [ 14,  22,  14],
         ...,
         [118, 147, 212],
         [118, 147, 212],
         [118, 146, 211]],
 
        [[  0,   0,   0],
         [  9,  14,   9],
         [  9,  14,   9],
         ...,
         [118, 147, 211],
         [118, 146, 211],
         [118, 146, 210]],
 
        ...,
 
        [[ 74, 101,  72],
         [ 78, 107,  76],
         [ 87, 119,  85],
         ...,
         [ 65,  89,  64],
         [ 69,  95,  67],
         [ 78, 106,  76]],
 
        [[ 78, 107,  76],
         [ 87, 119,  85],
         [ 87, 119,  85],
         ...,
         [ 65,  89,  64],
         [ 65,  89,  64],
         [ 69,  95,  67]],
 
        [[ 87, 119,  85],
         [ 87, 119,  85],
         [ 77, 106,  76],
         ...,
         [ 73,  99,  

In [ ]:
!python3 test.py

# Run your agent
As the code below runs you should see episode videos and rewards show up. You can run the below cell multiple times to see different episodes.

In [ ]:
from time import time
action_centroids = np.load(TEST_KMEANS_MODEL_NAME)
network = NatureCNN((3, 64, 64), NUM_ACTION_CENTROIDS).cuda()
network.load_state_dict(th.load(TEST_MODEL_NAME))

stats = {'runtime': [], 'reward': []}
num_actions = action_centroids.shape[0]
action_list = np.arange(num_actions)

for episode in range(TEST_EPISODES):
    start = time()
    obs = env.reset()
    done = False
    total_reward = 0
    steps = 0

    while not done:
        # Process the action:
        #   - Add/remove batch dimensions
        #   - Transpose image (needs to be channels-last)
        #   - Normalize image
        obs = th.from_numpy(obs['pov'].transpose(2, 0, 1)[None].astype(np.float32) / 255).cuda()
        # Turn logits into probabilities
        probabilities = th.softmax(network(obs), dim=1)[0]
        # Into numpy
        probabilities = probabilities.detach().cpu().numpy()
        # Sample action according to the probabilities
        discrete_action = np.random.choice(action_list, p=probabilities)

        # Map the discrete action to the corresponding action centroid (vector)
        action = action_centroids[discrete_action]
        minerl_action = {"vector": action}

        obs, reward, done, info = env.step(minerl_action)
        total_reward += reward
        steps += 1
        if steps >= MAX_TEST_EPISODE_LEN:
            break

    stats['runtime'].append(time() - start)
    stats['reward'].append(total_reward)

    # env.release()
    # env.play()
    print(f'Episode #{episode + 1} reward: {total_reward}\t\t episode length: {steps}\n')

Episode #1 reward: 0.0		 episode length: 1963



KeyboardInterrupt: ignored

In [ ]:
# import json
# with open('stats_BC.json', 'w') as outfile:
#     json.dump(stats, outfile)